In [1]:
from hmr4d import os_chdir_to_proj_root
import os

os_chdir_to_proj_root()
os.environ["PYOPENGL_PLATFORM"] = "egl"

In [2]:
import torch
from pathlib import Path
from hmr4d.utils.wis3d_utils import convert_motion_as_line_mesh, draw_T_w2c
import pyrender
import trimesh

# Load HML3D example data (motino + text)
joints3d = torch.load("inputs/hml3d/joints3d.pth", weights_only=False)

/tmp/ipykernel_3167621/2274608012.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  joints3d = torch.load("inputs/hml3d/joints3d.pth")


In [3]:
from hmr4d.utils.geo.hmr_cam import create_camera_sensor

# Data
seq_name = "000000"
j3d = joints3d[f"{seq_name}.npy"]["joints3d"]  # (T, 22, 3)

# Render
j3d_0offset = j3d - j3d[:, [0], :]
vertices, faces, vertex_colors = convert_motion_as_line_mesh(j3d_0offset)

In [17]:
# Pytorch3d
import cv2
import numpy as np
from hmr4d.utils.vis.renderer_utils import simple_render_mesh_background, simple_render_mesh
from hmr4d.utils.video_io_utils import save_video


if True:  # Pytorch3d doesn't work as requirements are too old (torch>=2.0)
    H, W = 1024, 1024
    K = create_camera_sensor(W, H, 24)[2]
    focal_length = K[0, 0]  # 24mm lens

    verts_render = vertices + torch.tensor([0.0, 0.0, 2.0])
    verts_render = verts_render * torch.tensor([1, -1, 1])
    render_dict = {
        "whf": (W, H, focal_length),
        "faces": faces,
        "verts": verts_render,
    }
    images = simple_render_mesh(render_dict, VI=1)

images = [cv2.resize(img, dsize=np.array([512, 512])) for img in images]

save_video(images, "tmp.mp4")

Rendering: 100%|██████████| 117/117 [00:02<00:00, 51.16it/s]


In [60]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from hmr4d.utils.vis.pyrender_utils import get_campose_pyrender
from tqdm import tqdm

width, height = 512, 512

rendered_imgs = []
for i in tqdm(range(vertices.shape[0])):
    scene = pyrender.Scene()

    # Add Person
    example_trimesh = trimesh.Trimesh(vertices=vertices[i], faces=faces)
    mesh = pyrender.Mesh.from_trimesh(example_trimesh)
    scene.add(mesh)

    # Add Camera
    cam = pyrender.PerspectiveCamera(yfov=(np.pi / 3.0))
    p_from = np.array([0.0, 0.2, 2.0])
    p_to = np.array([0.0, 0.0, 0.0])
    gravity_vec = np.array([0.0, -1.0, 0.0])
    cam_pose = get_campose_pyrender(p_from, p_to, gravity_vec)
    scene.add(cam, pose=cam_pose)

    # Add Light
    light = pyrender.SpotLight(
        color=np.array([0.8, 0.8, 0.8]), intensity=5.0, innerConeAngle=np.pi / 16.0, outerConeAngle=np.pi / 6.0
    )
    scene.add(light, pose=cam_pose)
    color, depth = pyrender.OffscreenRenderer(width, height).render(scene)
    # Image.fromarray(color)
    rendered_imgs.append(color)

if False:
    from hmr4d.utils.wis3d_utils import make_wis3d, add_motion_as_lines

    wis3d = make_wis3d(name="debug_pyrender")
    T_w2c = torch.from_numpy(cam_pose)[None].float()
    T_w2c = T_w2c.inverse()
    draw_T_w2c(wis3d, T_w2c, name="debug_pyrender")
    add_motion_as_lines(j3d_0offset[:1], wis3d, name="j3d")

100%|██████████| 117/117 [00:27<00:00,  4.25it/s]


In [61]:
from hmr4d.utils.video_io_utils import save_video

save_video(rendered_imgs, "tmp.mp4")